In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io as io
import thinplate as tps
import cv2
import skimage
import dlib
from skimage.color import rgb2gray,rgb2lab
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
from skimage.morphology import disk
from skimage.filters import rank

In [2]:
import numpy as np

class TPS:       
    @staticmethod
    def fit(c, lambd=0., reduced=False):        
        n = c.shape[0]

        U = TPS.u(TPS.d(c, c))
        K = U + np.eye(n, dtype=np.float32)*lambd

        P = np.ones((n, 3), dtype=np.float32)
        P[:, 1:] = c[:, :2]

        v = np.zeros(n+3, dtype=np.float32)
        v[:n] = c[:, -1]

        A = np.zeros((n+3, n+3), dtype=np.float32)
        A[:n, :n] = K
        A[:n, -3:] = P
        A[-3:, :n] = P.T

        theta = np.linalg.solve(A, v) # p has structure w,a
        return theta[1:] if reduced else theta
        
    @staticmethod
    def d(a, b):
        return np.sqrt(np.square(a[:, None, :2] - b[None, :, :2]).sum(-1))

    @staticmethod
    def u(r):
        return r**2 * np.log(r + 1e-6)

    @staticmethod
    def z(x, c, theta):
        x = np.atleast_2d(x)
        U = TPS.u(TPS.d(x, c))
        w, a = theta[:-3], theta[-3:]
        reduced = theta.shape[0] == c.shape[0] + 2
        if reduced:
            w = np.concatenate((-np.sum(w, keepdims=True), w))
        b = np.dot(U, w)
        return a[0] + a[1]*x[:, 0] + a[2]*x[:, 1] + b

def uniform_grid(shape):
    '''Uniform grid coordinates.
    
    Params
    ------
    shape : tuple
        HxW defining the number of height and width dimension of the grid

    Returns
    -------
    points: HxWx2 tensor
        Grid coordinates over [0,1] normalized image range.
    '''

    H,W = shape[:2]    
    c = np.empty((H, W, 2))
    c[..., 0] = np.linspace(0, 1, W, dtype=np.float32)
    c[..., 1] = np.expand_dims(np.linspace(0, 1, H, dtype=np.float32), -1)

    return c
    
def tps_theta_from_points(c_src, c_dst, reduced=False):
    delta = c_src - c_dst
    
    cx = np.column_stack((c_dst, delta[:, 0]))
    cy = np.column_stack((c_dst, delta[:, 1]))
        
    theta_dx = TPS.fit(cx, reduced=reduced)
    theta_dy = TPS.fit(cy, reduced=reduced)

    return np.stack((theta_dx, theta_dy), -1)


def tps_grid(theta, c_dst, dshape):    
    ugrid = uniform_grid(dshape)

    reduced = c_dst.shape[0] + 2 == theta.shape[0]

    dx = TPS.z(ugrid.reshape((-1, 2)), c_dst, theta[:, 0]).reshape(dshape[:2])
    dy = TPS.z(ugrid.reshape((-1, 2)), c_dst, theta[:, 1]).reshape(dshape[:2])
    dgrid = np.stack((dx, dy), -1)

    grid = dgrid + ugrid
    
    return grid # H'xW'x2 grid[i,j] in range [0..1]

def tps_grid_to_remap(grid, sshape):
    '''Convert a dense grid to OpenCV's remap compatible maps.
    
    Params
    ------
    grid : HxWx2 array
        Normalized flow field coordinates as computed by compute_densegrid.
    sshape : tuple
        Height and width of source image in pixels.


    Returns
    -------
    mapx : HxW array
    mapy : HxW array
    '''

    mx = (grid[:, :, 0] * sshape[1]).astype(np.float32)
    my = (grid[:, :, 1] * sshape[0]).astype(np.float32)

    return mx, my

In [3]:
def warp_image_cv(img, c_src, c_dst, dshape=None):
    dshape = dshape or img.shape
    theta = tps_theta_from_points(c_src, c_dst, reduced=True)
    grid = tps_grid(theta, c_dst, dshape)
    mapx, mapy = tps_grid_to_remap(grid, img.shape)
    return cv2.remap(img, mapx, mapy, cv2.INTER_CUBIC)

In [4]:
def face_detection(image):
    img = np.copy(image)
    # set up the 68 point facial landmark detector
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    # bring in the input image
    # img = cv2.imread('maxresdefault.jpg', 1)
    
    # convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect faces in the image
    faces_in_image = detector(img_gray, 0)
    # loop through each face in image
    for face in faces_in_image:

        # assign the facial landmarks
        landmarksref = predictor(img_gray, face)

        # unpack the 68 landmark coordinates from the dlib object into a list 
        landmarks_listref = []
        v=[
            [0,0],[1,0],[1,1],[0,1]
        ]
        for i in range(0, landmarksref.num_parts):
            landmarks_listref.append((landmarksref.part(i).x, landmarksref.part(i).y))
            v.append([(landmarksref.part(i).x)/img.shape[0], (landmarksref.part(i).y)/img.shape[1]])

        # for each landmark, plot and write number
        for landmark_numref, xy in enumerate(landmarks_listref, start = 1):
            cv2.circle(img, (xy[0], xy[1]), 12, (168, 0, 20), -1)
            cv2.putText(img, str(landmark_numref),(xy[0]-7,xy[1]+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(255,255,255), 1)
            
    return landmarks_listref, v
        

In [5]:
def crop_face(img_original):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = np.copy(img_original)
    rect = detector(img)[0]
    sp = predictor(img, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])

    left_eye = landmarks[[*range(38,35,-1), *range(41,38,-1)]]
    right_eye = landmarks[[*range(44,41,-1), *range(47,44,-1)]]
    libs = landmarks[[*range(54,47,-1), *range(60,54,-1)]]
    outline = landmarks[[*range(17), *range(26,16,-1)]]

#     print(landmarks,outline)
    
    Y, X = skimage.draw.polygon(outline[:,1], outline[:,0])
    # they are colored
    Ylibs, Xlibs = skimage.draw.polygon(libs[:,1], libs[:,0])
    Yle, Xle = skimage.draw.polygon(left_eye[:,1], left_eye[:,0])
    Yre, Xre = skimage.draw.polygon(right_eye[:,1], right_eye[:,0])

    cropped_img = np.zeros(img.shape, dtype=np.uint8)
    # for libs and eyes
    cropped_img_libs = np.zeros(img.shape, dtype=np.uint8)
    cropped_img_leye = np.zeros(img.shape, dtype=np.uint8)
    cropped_img_reye = np.zeros(img.shape, dtype=np.uint8)
    # the line below means that we copy the shape(face) for the region of X and Y, to the black image(cropped_img)
#     cropped_img[Y, X] = img[Y, X]
    img[Ylibs, Xlibs] = cropped_img_libs[Ylibs, Xlibs]
    img[Yle, Xle] = cropped_img_leye[Yle, Xle]
    img[Yre, Xre] = cropped_img_reye[Yre, Xre]
    cropped_img[Y, X] = img[Y, X]
    img = img_original-cropped_img
    return cropped_img,img

In [6]:
def crop_libs(img_original):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = np.copy(img_original)
    rect = detector(img)[0]
    sp = predictor(img, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])

    libs = landmarks[[*range(54,47,-1), *range(60,54,-1)]]

    Ylibs, Xlibs = skimage.draw.polygon(libs[:,1], libs[:,0])

    cropped_img_libs = np.zeros(img.shape, dtype=np.uint8)
    # the line below means that we copy the shape(face) for the region of X and Y, to the black image(cropped_img)
    
    cropped_img_libs[Ylibs, Xlibs] = img[Ylibs, Xlibs]
    
    return cropped_img_libs

In [7]:
def wlsfilter_layer(image_orig, beta=0.2 ,lambda_=0.2):

    image = image_orig.astype(np.float)/255.0
    image1 = image.flatten(1)
    s = image.shape
    k = s[0]*s[1]

    dy = np.diff(image, 1, 0)
    dx = np.diff(image, 1, 1)
    
    dy = -beta*lambda_ / ((np.absolute(dy) ** 1.2 + 0.0001))
    dx = -beta*lambda_ / ((np.absolute(dx) ** 1.2 + 0.0001))


    dy = np.vstack((dy, np.zeros(s[1], )))
    dx = np.hstack((dx, np.zeros(s[0], )[:, np.newaxis]))

    dy = dy.flatten(1)
    dx = dx.flatten(1)
    
    d = 1 - (dx + np.roll(dx, s[0]) + dy + np.roll(dy, 1))

    a = spdiags(np.vstack((dx, dy)), [-s[0], -1], k, k)
    a = a + a.T + spdiags(d, 0, k, k)

    temp = spsolve(a, image1).reshape(s[::-1])
    
    base = np.rollaxis(temp,1)
    detail = image - base
    return (base), (detail)

In [8]:
# show images
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [9]:
# imgr = cv2.imread('examples/80811699_418066922406466_6994441794444328960_n1.jpg', 1)
# imgt = cv2.imread('input/79436089_822552084866705_868234996165378048_n.jpg', 1)

# reference(with makeup)
imgr = cv2.imread('detectedref.jpg', 1)
# target(without makeup)
imgt = cv2.imread('input/image1.png', 1)

dim = (600, 600)
imgr = cv2.resize(imgr, dim)

dim = (600, 600)
imgt = cv2.resize(imgt, dim)

landmarkref,vref = face_detection(imgr)
landmarktar,vtar = face_detection(imgt)

# show_images([imgr, imgt], ['reference', 'target'])

v_example = np.array(vref)
v_input = np.array(vtar)

veoutline = v_example[[*range(17), *range(26,16,-1)]]
vioutline = v_input[[*range(17), *range(26,16,-1)]]

wapred_img = warp_image_cv(imgr, veoutline, vioutline)

# show_images([wapred_img], ['warped'])

croppedtar_face , back_ground = crop_face(imgt)
croppedref_face , back_groundref = crop_face(wapred_img)

# show_images([croppedtar_face, croppedref_face], ['cropped face target', 'cropped face refernece'])
# show_images([back_ground, back_groundref], ['background target', 'background reference'])

croppedtar_lips = crop_libs(imgt)
croppedref_lips = crop_libs(wapred_img)

# show_images([croppedtar_lips, croppedref_lips], ['cropped lips target', 'cropped lips refernece'])

imager_face = cv2.cvtColor(croppedref_face, cv2.COLOR_BGR2LAB)
imaget_face = cv2.cvtColor(croppedtar_face, cv2.COLOR_BGR2LAB)

cv2.imwrite('imager_face.jpg',imager_face)
cv2.imwrite('imaget_face.jpg',imaget_face)

imager_lip = cv2.cvtColor(croppedref_lips, cv2.COLOR_BGR2LAB)
imaget_lip = cv2.cvtColor(croppedtar_lips, cv2.COLOR_BGR2LAB)

cv2.imwrite('imager_lip.jpg',imager_lip)
cv2.imwrite('imaget_lip.jpg',imaget_lip)

LS_tar,D_tar = wlsfilter_layer (imaget_face[:,:,0])
LS_ref,D_ref = wlsfilter_layer (imager_face[:,:,0])

LS_tar_lip,D_tar_lip = wlsfilter_layer (imaget_lip[:,:,0])
LS_ref_lip,D_ref_lip = wlsfilter_layer (imager_lip[:,:,0])

D_res = D_ref + 0.5*D_tar
D_res_lip = D_ref_lip + 0.5*D_tar_lip

Rc_A = 0.2*imaget_face[:,:,1] + 0.8*imager_face[:,:,1]
Rc_B = 0.2*imaget_face[:,:,2] + 0.8*imager_face[:,:,2]

LIP_A = 0.2*imaget_lip[:,:,1] + 0.8*imager_lip[:,:,1]
LIP_B = 0.2*imaget_lip[:,:,1] + 0.8*imager_lip[:,:,1]


Rsum = imaget_face.copy() 
Rsum1 = imaget_lip.copy()

Rsum[:,:,0] = (LS_tar + D_res)*255.0
Rsum[:,:,1] = Rc_A 
Rsum[:,:,2] = Rc_B 

Rsum1[:,:,0] = (D_res_lip + LS_tar_lip)*255.0
Rsum1[:,:,1] = LIP_A
Rsum1[:,:,2] = LIP_B

R = cv2.cvtColor(Rsum, cv2.COLOR_LAB2BGR)

R1 = cv2.cvtColor(Rsum1, cv2.COLOR_LAB2BGR)

result = R + R1
# show_images([R, R1, result], ['resulted face', 'resulted lips', 'result cropped face'])

background = back_ground-croppedtar_lips

cv2.imwrite('result.jpg',result)
cv2.imwrite('background.jpg',background)

My_img = result + background

# show_images([My_img], ['result output'])

cv2.imshow("outImg", My_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

cv2.imwrite('outImg.jpg', My_img)

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  after removing the cwd from sys.path.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


True